In [1]:
import pandas as pd
import os

In [2]:
data = '../data/qualtrics_data/'

In [3]:
files = os.listdir(data)

In [4]:
for file in files:
    if os.path.isfile(data+file):
        print(file)

qualtrics_ratings_long.csv
qualtrics_ratings_yuning.csv
qualtrics_ratings_wide.csv
qualtrics_ratings_wes.csv


In [5]:
df_dict = {}
for i in files:
    if os.path.isfile(data+i):
        df_dict[i] = pd.read_csv(data + i)
        if 'Unnamed: 0' in df_dict[i].columns:
            df_dict[i] = df_dict[i].drop('Unnamed: 0', axis=1)
sample1 = list(df_dict.values())[0]
sample2 = list(df_dict.values())[1]

In [6]:
def wide_to_long(df, name):
    return pd.melt(df, id_vars = sample1.columns[0:3], value_vars = sample1.columns[3:], value_name = name)


In [7]:
dataframe = pd.DataFrame(columns = ['id', 'section', 'summary', 'variable', 'value'])
sample = df_dict['qualtrics_ratings_yuning.csv']
yuning = wide_to_long(sample1, 'qualtrics_ratings_yuning.csv')
wes = wide_to_long(sample2, 'qualtrics_ratings_wes.csv')

dataframe = pd.merge(yuning, wes, on=['id', 'summary', 'variable'], how='outer')

dataframe = dataframe[['id', 'summary', 'section_x', 'variable',
       'qualtrics_ratings_yuning.csv',
       'qualtrics_ratings_wes.csv']]

# for i in df_dict:
#     temp_df = wide_to_long(df_dict[i], i)
#     dataframe = pd.merge(dataframe, temp_df, on=['id', 'section', 'summary', 'variable'], how='outer')
    


/tmp/ipykernel_123/4172876682.py:2: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  return pd.melt(df, id_vars = sample1.columns[0:3], value_vars = sample1.columns[3:], value_name = name)


KeyError: "The following 'id_vars' are not present in the DataFrame: ['section_x']"

In [ ]:
dataframe

In [ ]:
dataframe.to_csv('../results/qualtrics_ratings_long.csv')

In [5]:
dataframe = pd.read_csv('../data/qualtrics_data/qualtrics_ratings_long.csv', low_memory=False).dropna()
dataframe

,id,summary,section_x,variable,qualtrics_ratings_yuning.csv,qualtrics_ratings_wes.csv,Unnamed: 6
0,60109ec41bb0de422378c3012,2,microeconomics is based on individual and hous...,Voice,3,2,1
1,632aaef27780d596dc7e0a9f2,2,economics is concerned with the well-being of ...,Voice,3,2,1
2,6361a8a745dd1d5d6f192d362,2,economics is concerned with the well-being of ...,Voice,3,2,1
3,63c3343c5a670ddd37e82f2b2,2,economics is concerned with the well-being of ...,Voice,3,2,1
4,63cc61c7c2ae0809f0c71cb52,2,to understand why both microeconomic and macro...,Voice,3,2,1
...,...,...,...,...,...,...,...
733,63b5681c49af1cbe651420bc,4,"economies are complex systems, but we can use ...",Language,3,3,0
734,63b63938dbf229f3eb46d4a4,4,there are three common ways in which an econom...,Language,3,3,0
735,63cbfaf6ab54f6b8d8a054d7,4,this introduction describes the different type...,Language,3,3,0
736,63dae36f736faa53f7c68127,4,the modern economy is a complex system that in...,Language,3,3,0


In [7]:

from sklearn.metrics import cohen_kappa_score
import numpy as np

for i in dataframe['variable'].unique():
    wes_list = dataframe[dataframe['variable'] == i]['qualtrics_ratings_yuning.csv']
    yuning_list = dataframe[dataframe['variable'] == i]['qualtrics_ratings_wes.csv']
    print(i + ','+ str(cohen_kappa_score(np.array(yuning_list), np.array(wes_list), weights='quadratic')))

wes_list = dataframe['qualtrics_ratings_wes.csv']
yuning_list = dataframe['qualtrics_ratings_yuning.csv']
str(cohen_kappa_score(np.array(yuning_list), np.array(wes_list), weights='quadratic'))

Voice,0.31998194742186625
Organization,0.4849578045480899
Language ,0.5313648115389848
Main Points,0.5669272542817927
Details,0.6111837802485285
Wording,0.7296197558580049


'0.5758036759138156'

In [10]:
dataframe['adjudicated_score'] = (dataframe['qualtrics_ratings_yuning.csv']+ dataframe['qualtrics_ratings_wes.csv'])/2

In [16]:
wide_df = dataframe.pivot(index=['id', 'summary', 'section_x'], columns='variable', values='adjudicated_score').reset_index()

In [18]:
wide_df.columns = ['id', 'section', 'summary', 'Details', 'Language ', 'Main Points',
       'Organization', 'Voice', 'Wording']

In [20]:
wide_df.to_csv('../data/qualtrics_data/qualtrics_ratings_wide.csv')